In [ ]:
import ee
import requests
import shutil, sys

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
import logging
import multiprocessing
from retry import retry

In [ ]:
Berlin = ee.Image("users/Temporary/Copernicus_Land_Monitoring_Service/DE001L1_BERLIN_UA2012_DHM_v020")
London = ee.Image("users/Temporary/Copernicus_Land_Monitoring_Service/UK001L2_LONDON_UA2012_DHM_v020")

In [ ]:
img = London
img_geo = img.geometry()
img2 = ee.Image(0).clip(img_geo)
pointsROI = img2.sample(
      numPixels=20000,
      region=img_geo,
      scale=1120,
      geometries=True )

In [ ]:
print(pointsROI.size().getInfo())

4636


In [ ]:
S1Collection = (ee.ImageCollection("COPERNICUS/S1_GRD")
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VH'))
                    .select('VV','VH'))

In [ ]:
VIIRSCollection = (ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')
                  .select(['avg_rad'],['NL']))

In [ ]:
S2Collection = ee.ImageCollection("COPERNICUS/S2")

In [ ]:
terrain = ee.call('Terrain', ee.Image('USGS/SRTMGL1_003'))
elevation = terrain.select(['elevation'])

In [ ]:
imgS2 = (S2Collection.filterDate('2020-01-01','2021-01-01')
                                 .filterBounds(img_geo)
                                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))
                                 .select('B[2-8]','B8A','B1[1-2]')
                                 .median()
                                 )

imgS1_Asc = (S1Collection.filterDate('2020-01-01','2021-01-01')
                                 .filterBounds(img_geo)
                                 .filter(ee.Filter.eq('orbitProperties_pass','ASCENDING'))
                                 .mean().rename('VV_A','VH_A')
                                 )

imgS1_Desc = (S1Collection.filterDate('2020-01-01','2021-01-01')
                                 .filterBounds(img_geo)
                                 .filter(ee.Filter.eq('orbitProperties_pass','DESCENDING'))
                                 .mean().rename('VV_D','VH_D')
                                 )


imgNL = (VIIRSCollection.filterDate('2020-01-01','2021-01-01')
                            .filterBounds(img_geo)
                            .mean())

#imgNL_post = (VIIRSCollection.filterDate('2023-02-06','2023-02-11')
#                            .filterBounds(geometry)
#                            .mean().clip(geometry))

In [ ]:
InputImg = ee.Image.cat([imgS2, imgS1_Asc, imgS1_Desc,  imgNL, elevation])

In [ ]:
@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
    ID = point['id']
    point = ee.Geometry.Point(point['geometry']['coordinates'])
    ROI = point.buffer(112*10).bounds()

    imageInput = InputImg.clip(ROI)

  # Fetch the URL from which to download the image.
    url = imageInput.float().getDownloadUrl({
   #     'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
   #   'region': ROI,
        'scale': 10,
        'dimensions': '224x224',
        'format': 'GEO_TIFF'
    })

    #print(url)

    r = requests.get(url)

    filename = '/dir/London/tile_%s.tif' % (ID)

    with open(filename, 'wb') as out_file:
        out_file.write(r.content)
    print("Done: ", index)


In [ ]:
items2 = pointsROI.toList(pointsROI.size().getInfo()).getInfo()
#items2 = pointsROI.toList(1).getInfo()
pool = multiprocessing.Pool(32)
pool.starmap(getResult, enumerate(items2))
pool.close()
pool.join()

Done:  518
Done:  851
Done:  111
Done:  74
Done:  925
Done:  481
Done:  999
Done:  407
Done:  777
Done:  814
Done:  185
Done:  148
Done:  962
Done:  1110
Done:  888
Done:  370
Done:  37
Done:  296
Done:  259
Done:  629
Done:  592
Done:  222
Done:  444
Done:  703
Done:  555
Done:  0
Done:  1073
Done:  1036
Done:  1147
Done:  740
Done:  666
Done:  333
Done:  186
Done:  75
Done:  260
Done:  630
Done:  704
Done:  1111
Done:  112
Done:  297
Done:  1
Done:  445
Done:  187
Done:  593
Done:  482
Done:  1112
Done:  446
Done:  1074
Done:  408
Done:  1037
Done:  889
Done:  38
Done:  963
Done:  1148
Done:  667
Done:  76
Done:  815
Done:  1000
Done:  Done: 261 
Done: 556 Done: 
926 
Done: 778
 334
Done:  519
Done:  223
Done:  149
Done:  852
Done:  371
Done:  631
Done:  705
Done:  113
Done:  594
Done:  741
Done:  409
Done:  298
Done:  188
Done:  2
Done:  483
Done:  447
Done:  1075
Done:  77
Done:  890
Done:  520
Done:  816
Done:  262
Done:  1113
Done:  335
Done:  39
Done:  372
Done:  1001
Done:  706